In [ ]:
'''Same as DataProcessing_long, use for batches of files'''
#import pd, np, signal for data processing
import pandas as pd
import numpy as np
from scipy import signal
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def remove_repeat(file_ls,svd_fil):
  #try to read in the saved data
  try:
    saved_s=pd.read_csv(svd_fil,index_col=0,header=None,squeeze=True)
  except FileNotFoundError:
    print('History file not found, all data files being processed')
    return file_ls,False
  
  #somewhat clunky (but effective) method for checking if we've processed a file already
  temp_s=pd.Series(file_ls)
  tf_ls=temp_s.isin(saved_s)
  idx_ls=[]
  for i in range(0,len(tf_ls)):
    if tf_ls[i]:
      idx_ls.append(i)
  temp_s.drop(index=idx_ls,inplace=True)
  return temp_s.tolist(),True

In [ ]:
#set filepaths for import/export

#ONLY PROCESS ONE FILE PATH WHEN RUNNING THE PROGRAM - comment the other
#file in path for synthetic data
#fin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Synthetic Data/Basic Synthetic Data/'
#file in path for raw data
fin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Raw Data/'

cwtout_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Continuous Wavelet Transformation/'
pkout_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/'
saved_data=r'/content/drive/My Drive/ML Spectroscopy/Programs/Data Processing/Saved Lists/processed_files.csv'

'''list of base filenames, note marbledslab, and granitegrid[...] 
require clustering for labels (hence 'None'). blackmin[...] excluded due to a 
lack of known label'''

#Find all files in data directory
from os import listdir
from os.path import isfile, join
fname_ls = [f for f in listdir(fin_path) if isfile(join(fin_path,f))]

fname_ls,fflag=remove_repeat(file_ls=fname_ls,svd_fil=saved_data)

if not fname_ls:
  raise ValueError('No new files')
#define the training labels in a dict - labels taken from "0-Uploaded Data" sheet 
t_labels={
    'qtz':0,
    'albite':1,
    'hb':2,
    'bt':3,
    'ms':4,
    'fo':5,
    'aug':6,
    'en':7,
    'an':8,
    'mc':9,
    'cal':10,
    'gyp':11,
    'qtzalb':12,
    'foaug':13,
    'foalb':14
}

fname_d={}

for fil in fname_ls:
  try:
    fname_d[fil]=t_labels[fil.split('_')[0]]
  except:
    fname_d[fil]=None





ValueError: ignored

In [ ]:
print(fname_d)

{'qtz_train_003s_4760_ds2.csv': 0, 'qtz_train_003s_4760_ds4.csv': 0, 'qtz_train_003s_4760_r.csv': 0, 'qtz_train_003s_4760_r_ds2.csv': 0, 'qtz_train_003s_4760_r_ds4.csv': 0, 'qtz_train_003s_1148_ds2.csv': 0, 'qtz_train_003s_1148_ds4.csv': 0, 'qtz_train_003s_1148_r.csv': 0, 'qtz_train_003s_1148_r_ds4.csv': 0, 'qtz_train_003s_1148_r_ds2.csv': 0, 'qtz_train_003s-1599_ds2.csv': 0, 'qtz_train_003s-1599_ds4.csv': 0, 'qtz_train_003s-1599_r.csv': 0, 'qtz_train_003s-1599_r_ds2.csv': 0, 'qtz_train_003s-1599_r_ds4.csv': 0, 'qtz_train_005s_0900_ds2.csv': 0, 'qtz_train_005s_0900_ds4.csv': 0, 'qtz_train_005s_0900_r.csv': 0, 'qtz_train_005s_0900_r_ds2.csv': 0, 'qtz_train_005s_0900_r_ds4.csv': 0, 'albite_train_015s_5250_ds4.csv': 1, 'albite_train_015s_5250_ds2.csv': 1, 'albite_train_015s_5250_r.csv': 1, 'albite_train_015s_5250_r_ds2.csv': 1, 'albite_train_015s_5250_r_ds4.csv': 1, 'qtz_train_005s_1000_ds2.csv': 0, 'qtz_train_005s_1000_ds4.csv': 0, 'qtz_train_005s_1000_r.csv': 0, 'qtz_train_005s_1000_r_d

In [ ]:
for key,val in fname_d.items():
  if key.split('.')[-1]=='txt':
    df=pd.read_csv(fin_path+key,delim_whitespace=True)
    if df.shape[1]<20:
      df=pd.read_csv(fin_path+key,sep='\t')
  else:
    df=pd.read_csv(fin_path+key,delim_whitespace=False)
  df.reset_index(inplace=True)

  #create traceable index
  df['fname']=key.split('.')[0]
  df['og-idx']=df.fname+"-"+df.index.map(str)
  df.set_index(keys='og-idx',drop=True,inplace=True)
  
  #drop non-numeric columns from data
  dropcolumns=[]
  for i in df.columns.values:
    try:
      float(i)
    except ValueError:
      dropcolumns.append(i)

  df.drop(columns=dropcolumns,inplace=True)

  #trim to cols to [150,1100]
  df.drop(columns=[i for i in df.columns.values if float(i)<150.],inplace=True)
  df.drop(columns=[i for i in df.columns.values if float(i)>1100.],inplace=True)
  
  #standardize wavelength
  new_wavelength=1
  starting_wave=150
  ending_wave=1100
  std_df=pd.DataFrame()

  for i in range(starting_wave,ending_wave,new_wavelength):
    std_df[i]=df[[j for j in df.columns.values if i<=float(j)<(i+new_wavelength)]].min(axis=1)

  #implement cwt
  c_vals=std_df.columns.values.tolist()
  wvlt=signal.ricker
  smth_df=pd.DataFrame(columns=c_vals)

  for idx, row in std_df.iterrows():
    smth_df=smth_df.append(other=pd.Series(signal.cwt(data=row,widths=[3],wavelet=wvlt)[0,:],name=idx,index=c_vals))

  #scale df to [-1,1]
  scl_df=smth_df.divide(smth_df.max(1),axis=0)

  #find peaks
  pk_ls=[['peak1','peak2','peak3','peak4','peak5','val1','val2','val3','val4','val5']]
  for idx,row in std_df.iterrows():
    #print(idx,'\n',row,'\n\n')
    if row.sum()<100:
      continue
    pk_wv_ls=std_df.columns.values[signal.find_peaks_cwt(row,widths=[3],wavelet=wvlt)][1:-1] 
    temp_vals=pd.Series(scl_df.iloc[std_df.index.get_loc(std_df.loc[idx].name)][pk_wv_ls],index=pk_wv_ls)
    temp_s=temp_vals.nlargest(n=5)
    pk_wv=temp_s.index.values.tolist()
    pk_vl=temp_s.tolist()
    pk_ls.append(pk_wv + pk_vl)
  try:
    pk_df=pd.DataFrame(pk_ls[1:],index=std_df.index.values,columns=pk_ls[0])
    pk_flag=True
  except:
    pk_flag=False
  print()

  #check if data has a label, add if it exists

  if val is None:
    final_cwt_path=cwtout_path+r'Unlabeled/cwt_'+key
    final_pk_path=pkout_path+r'Unlabeled/pk_'+key
  else:
    scl_df['label']=val
    if pk_flag:
      pk_df['label']=val
    final_cwt_path=cwtout_path+r'Labeled/L_cwt_'+key
    final_pk_path=pkout_path+r'Labeled/L_pk_'+key


  #output files
  scl_df.to_csv(final_cwt_path)
  if pk_flag:
    pk_df.to_csv(final_pk_path)

  print(key,'labeled')



  if fflag:
    pd.Series(key).to_csv(saved_data,mode='a')
  else:
    pd.Series(key).to_csv(saved_data,mode='w')


/usr/local/lib/python3.6/dist-packages/scipy/signal/_peak_finding.py:77: RuntimeWarning: invalid value encountered in greater
  results &= comparator(main, plus)
/usr/local/lib/python3.6/dist-packages/scipy/signal/_peak_finding.py:78: RuntimeWarning: invalid value encountered in greater
  results &= comparator(main, minus)


IndexError: ignored